In [25]:
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms as transforms
import json
import os
import torch
from torchvision.datasets import VisionDataset
from torchvision.datasets.utils import check_integrity, download_and_extract_archive
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets,models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
batch_size = 4
client_aug = 2
augment= False
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(device)

cuda


In [26]:
import gc
gc.collect()
torch.cuda.empty_cache()

import warnings
warnings.filterwarnings('ignore')

In [27]:
train_dir = "/home/siu856542507/zarif/FL-Violence/Train"
test_dir = "/home/siu856542507/zarif/FL-Violence/Test"

SEED = 12
IMG_HEIGHT = 64
IMG_WIDTH = 64
BATCH_SIZE = 16    
EPOCHS = 30
LR =  0.00003
NUM_CLASSES = 14
CLASS_LABELS = ['Abuse','Arrest','Arson','Assault','Burglary','Explosion','Fighting',"Normal",'RoadAccidents','Robbery','Shooting','Shoplifting','Stealing','Vandalism']

crime_types=os.listdir(train_dir)
n=len(crime_types)
print("Number of crime categories : ",n)

Number of crime categories :  14


In [88]:
transform_train = transforms.Compose([
    transforms.Resize(32),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=30),
    transforms.ToTensor(),
    transforms.Normalize((0.3604, 0.3520, 0.3479),(0.229, 0.224, 0.225))
])


transform_test = transforms.Compose([
        transforms.Resize(32),
    transforms.ToTensor(),
])

train_dataset=datasets.ImageFolder(train_dir, transform=transform_train)
test_dataset=datasets.ImageFolder(test_dir,transform=transform_test)


# train_loader = torch.utils.data.DataLoader(
#         train_dataset,
#         batch_size=batch_size,
#         shuffle=True,
#         num_workers=0,
#     )
# test_loader =torch.utils.data.DataLoader(
#         test_dataset,
#         batch_size=batch_size,
#         shuffle=False,
#         num_workers=0,
#     )



In [146]:
from collections import Counter
label_ct = dict(Counter(train_dataset.targets))
print(dict(Counter(train_dataset.targets)))

{0: 19076, 1: 26397, 2: 24421, 3: 10360, 4: 39504, 5: 18753, 6: 24684, 7: 947768, 8: 23486, 9: 41493, 10: 7140, 11: 24835, 12: 44802, 13: 13626}


In [150]:
class_weights = []
for key in label_ct:
    class_weights.append(1/label_ct[key])
class_weights

[5.242189138184106e-05,
 3.788309277569421e-05,
 4.0948364112853693e-05,
 9.652509652509652e-05,
 2.5313892264074525e-05,
 5.332480136511492e-05,
 4.0512072597634096e-05,
 1.0551105333794767e-06,
 4.2578557438473986e-05,
 2.4100450678427685e-05,
 0.00014005602240896358,
 4.02657539762432e-05,
 2.2320432123565913e-05,
 7.338910905621606e-05]

In [90]:
# mean = 0.
# std = 0.
# for images, _ in train_loader:
#     batch_samples = images.size(0) # batch size (the last batch can have smaller size!)
#     images = images.view(batch_samples, images.size(1), -1)
#     mean += images.mean(2).sum(0)
#     std += images.std(2).sum(0)

# mean /= len(train_loader.dataset)
# std /= len(train_loader.dataset)

In [91]:
# print ("mean: ",mean)
# print ("std: ",std)

# mean:  tensor([0.3604, 0.3520, 0.3479])
# std:  tensor([0.2343, 0.2329, 0.2372])

In [92]:
# normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                  std=[0.229, 0.224, 0.225])

In [186]:
transform = transforms.Compose([
    transforms.Resize(32),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=30),
    transforms.ToTensor(),
    transforms.Normalize((0.3604, 0.3520, 0.3479),(0.229, 0.224, 0.225))
])

train_dataset = datasets.ImageFolder(train_dir, transform=transform)
# train_dataset.classes = ['3','8']
# train_dataset.class_to_idx = {'3':0,'8':1}
# train_dataset.samples = list(filter(lambda s: s[1] in [0,1], train_dataset.samples))

# dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

In [187]:
import seaborn as sns
import plotly.express as px
import os
import numpy as np

fig = px.bar(x = CLASS_LABELS,
             y = [list(train_dataset.targets).count(i) for i in np.unique(train_dataset.targets)] , 
             color = np.unique(train_dataset.targets) ,
             color_continuous_scale="Emrld") 
fig.update_xaxes(title="Classes")
fig.update_yaxes(title = "Number of Images")
fig.update_layout(showlegend = True,
    title = {
        'text': 'Train Data Distribution ',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()



In [129]:
import torch
from torchvision import datasets
from torchvision import transforms
import matplotlib.pyplot as plt

# transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),


# def non_iid_split(dataset, nb_nodes, n_samples_per_node, batch_size, shuffle, shuffle_digits=False):
#     assert(nb_nodes>0 and nb_nodes<=10)

#     digits=torch.arange(10) if shuffle_digits==False else torch.randperm(10, generator=torch.Generator().manual_seed(0))
#     digits2=torch.arange(3) if shuffle_digits==False else torch.randperm(10, generator=torch.Generator().manual_seed(0))

#     digits = torch.cat((digits,digits2))

   
#     digits_split=list()
#     i=0
#     for n in range(nb_nodes, 0, -1):
#         inc=int((10-i)/n)
#         digits_split.append(digits[i:i+3])
#         print
#         i+=inc
#         print(inc)
#         print(digits_split)
    
#     print(digits_split)
#     # load and shuffle nb_nodes*n_samples_per_node from the dataset
#     loader = torch.utils.data.DataLoader(dataset,
#                                         batch_size=nb_nodes*n_samples_per_node,
#                                         shuffle=shuffle)
#     dataiter = iter(loader)
#     images_train_mnist, labels_train_mnist = next(dataiter)

#     data_splitted=list()
#     for i in range(nb_nodes):
#         idx=torch.stack([y_ == labels_train_mnist for y_ in digits_split[i]]).sum(0).bool() # get indices for the digits
#         data_splitted.append(torch.utils.data.DataLoader(torch.utils.data.TensorDataset(images_train_mnist[idx], labels_train_mnist[idx]), batch_size=batch_size, shuffle=shuffle))
#     print(data_splitted)
#     return data_splitted
def non_iid_split(dataset, nb_nodes, n_samples_per_node, batch_size, shuffle, shuffle_digits=False):
    assert(nb_nodes>0 and nb_nodes<=14)

    digits=torch.arange(14) if shuffle_digits==False else torch.randperm(14, generator=torch.Generator().manual_seed(0))

    # split the digits in a fair way
    digits_split=list()
    i=0
    for n in range(nb_nodes, 0, -1):
        inc=int((14-i)/n)
        digits_split.append(digits[i:i+inc])
        i+=inc

    # load and shuffle nb_nodes*n_samples_per_node from the dataset
    loader = torch.utils.data.DataLoader(dataset,
                                        batch_size=nb_nodes*n_samples_per_node,
                                        shuffle=shuffle)
    dataiter = iter(loader)
    images_train_mnist, labels_train_mnist =next(dataiter)

    data_splitted=list()
    for i in range(nb_nodes):
        idx=torch.stack([y_ == labels_train_mnist for y_ in digits_split[i]]).sum(0).bool() # get indices for the digits
        data_splitted.append(torch.utils.data.DataLoader(torch.utils.data.TensorDataset(images_train_mnist[idx], labels_train_mnist[idx]), batch_size=batch_size, shuffle=shuffle))

    return data_splitted

def iid_split(dataset, nb_nodes, n_samples_per_node, batch_size, shuffle):
    # load and shuffle n_samples_per_node from the dataset
    loader = torch.utils.data.DataLoader(dataset,
                                        batch_size=n_samples_per_node,
                                        shuffle=True)
    dataiter = iter(loader)

    data_splitted=list()

    for _ in range(nb_nodes):
        data_splitted.append(torch.utils.data.DataLoader(torch.utils.data.TensorDataset(*(next(dataiter))), batch_size=batch_size, shuffle=shuffle))

    return data_splitted


def  get_violence(type, n_samples_train, n_samples_test, n_clients, batch_size, shuffle):


        transform_train = transforms.Compose([
    transforms.Resize(32),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=30),
    transforms.ToTensor(),
    transforms.Normalize((0.3604, 0.3520, 0.3479),(0.229, 0.224, 0.225))
])


        transform_test = transforms.Compose([
                transforms.Resize(32),
            transforms.ToTensor(),
        ])

        train_dataset=datasets.ImageFolder(train_dir, transform=transforms.ToTensor())
        test_dataset=datasets.ImageFolder(test_dir,transform=transforms.ToTensor())

        if type=="iid":
            train=iid_split(train_dataset, n_clients, n_samples_train, batch_size, shuffle)
            test=iid_split(test_dataset, n_clients, n_samples_test, batch_size, shuffle)
        elif type=="non_iid":
            train=non_iid_split(train_dataset, n_clients, n_samples_train, batch_size, shuffle)
            test=non_iid_split(test_dataset, n_clients, n_samples_test, batch_size, shuffle)
        else:
            train=[]
            test=[]

        return train, test
    
def  get_MNIST(type="iid", n_samples_train=200, n_samples_test=100, n_clients=3, batch_size=25, shuffle=True):
    dataset_loaded_train = datasets.MNIST(
            root="./data",
            train=True,
            download=True,
            transform=transforms.ToTensor()
    )
    dataset_loaded_test = datasets.MNIST(
            root="./data",
            train=False,
            download=True,
            transform=transforms.ToTensor()
    )

    if type=="iid":
        train=iid_split(dataset_loaded_train, n_clients, n_samples_train, batch_size, shuffle)
        test=iid_split(dataset_loaded_test, n_clients, n_samples_test, batch_size, shuffle)
    elif type=="non_iid":
        train=non_iid_split(dataset_loaded_train, n_clients, n_samples_train, batch_size, shuffle)
        test=non_iid_split(dataset_loaded_test, n_clients, n_samples_test, batch_size, shuffle)
    else:
        train=[]
        test=[]

    return train, test
def plot_samples(data, channel:int, title=None, plot_name="", n_examples =20):

    n_rows = int(n_examples / 3)
    plt.figure(figsize=(1* n_rows, 1*n_rows))
    if title: plt.suptitle(title)
    X, y= data
    print(y)
    for idx in range(n_examples):
        
        ax = plt.subplot(n_rows, 5, idx + 1)

        image = 255 - X[idx, channel].view((64,64))
        ax.imshow(image, cmap='gist_gray')
        ax.axis("off")

    if plot_name!="":plt.savefig(f"plots/"+plot_name+".png")

    plt.tight_layout()

In [131]:
mnist_iid_train_dls, mnist_iid_test_dls = get_violence("non_iid",
    n_samples_train =500, n_samples_test=200, n_clients =10, 
    batch_size = batch_size, shuffle =True)

In [144]:
(len(mnist_iid_train_dls))

10

In [132]:
for i in range(len(mnist_iid_train_dls)):

    x,y=next(iter(mnist_iid_train_dls[i]))
    print(y)

tensor([0, 0, 0, 0])
tensor([1, 1, 1, 1])
tensor([2, 2, 2, 2])
tensor([3, 3, 3, 3])
tensor([4, 4, 4, 4])
tensor([5, 5, 5, 5])
tensor([7, 7, 7, 7])
tensor([9, 9, 8, 8])
tensor([11, 10, 10, 11])
tensor([12, 12, 12, 12])


tensor([12, 12, 12, 12])


In [135]:

# mnist_iid_train_dls, mnist_iid_test_dls = get_MNIST("iid",
#     n_samples_train =500, n_samples_test=100, n_clients =10, 
#     batch_size =batch_size, shuffle =True)

In [136]:
# from torchsummary import summary

# model = models.vgg19(pretrained=True)

# for param in model.parameters():
#     param.requires_grad = False

# # num_features = model.fc.in_features
# # model.fc = nn.Sequential(
# #     nn.Linear(num_features, 256),
# #     nn.ReLU(),
# #     nn.Linear(256, 128),
# #     nn.ReLU(),
# #     nn.Linear(128, 64),
# #     nn.ReLU(),
# #     nn.Dropout(p=0.5),
# #     nn.Linear(64, NUM_CLASSES)
# # )

# model.classifier[6] = nn.Sequential(
#                       nn.Linear(4096, 2048), 
#                       nn.ReLU(), 
#                       nn.Dropout(0.4),
#                       nn.Linear(2048, 1024),
#                       nn.ReLU(),
#                       nn.Dropout(0.2),
#                       nn.Linear(1024, 512),
#                       nn.ReLU(),
#                       nn.Dropout(0.1),
#                       nn.Linear(512,NUM_CLASSES),                   
#                       nn.LogSoftmax(dim=1))

# model.classifier

# # Transfer the model to device
# model = model.to(device)
# # Transfer the model to device


In [137]:
# from torchsummary import summary

# model = models.resnet50(pretrained = True)

# for param in model.parameters():
#     param.requires_grad = False

# num_features = model.fc.in_features
# print(num_features)
# model.fc = nn.Sequential(
#     nn.Linear(num_features, 256),
#     nn.ReLU(inplace=True),
#     nn.Linear(256, 128),
#     nn.ReLU(inplace=True),
#     nn.Linear(128, 64),
#     nn.ReLU(inplace=True),
#     nn.Linear(64, 10)
# )

# # Transfer the model to device
# model = model.to(device)
# Transfer the model to device


In [138]:
# Creating a CNN class
class ConvNeuralNet(nn.Module):
	#  Determine what layers and their order in CNN object 
    def __init__(self):
        super(ConvNeuralNet, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.conv_layer3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv_layer4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.fc1 = nn.Linear(10816, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 14)
    
    # Progresses data across layers    
    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)
        out = self.max_pool1(out)
        
        out = self.conv_layer3(out)
        out = self.conv_layer4(out)
        out = self.max_pool2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out
    
model = ConvNeuralNet().to(device)

In [139]:
# class MnistResNet(nn.Module):
#   def __init__(self, in_channels=1):
#     super(MnistResNet, self).__init__()

#     # Load a pretrained resnet model from torchvision.models in Pytorch
#     self.model = models.resnet50(pretrained=True)

#     # Change the input layer to take Grayscale image, instead of RGB images. 
#     # Hence in_channels is set as 1 or 3 respectively
#     # original definition of the first layer on the ResNet class
#     # self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
#     self.model.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
    
#     # Change the output layer to output 10 classes instead of 1000 classes
#     num_ftrs = self.model.fc.in_features
#     self.model.fc = nn.Linear(num_ftrs, 10)

#   def forward(self, x):
#     return self.model(x)


# my_resnet = MnistResNet()

# # input = torch.randn((16,1,244,244))
# # output = my_resnet(input)

# model = MnistResNet().to(device)


In [140]:
def loss_classifier(predictions,labels):
    
#     m = nn.LogSoftmax(dim=1)
    loss = nn.CrossEntropyLoss()
    
    return loss(predictions ,labels)


def loss_dataset(model, dataset, loss_f):
    """Compute the loss of `model` on `dataset`"""
    loss=0
    
    for idx,(features,labels) in enumerate(dataset):
        features = features.to(device)
        labels=labels.to(device)
        predictions= model(features)
        loss+=loss_f(predictions,labels)
    
    loss/=idx+1
    return loss


def accuracy_dataset(model, dataset):
    """Compute the accuracy of `model` on `dataset`"""
    
    correct=0
    
    for features,labels in iter(dataset):
        features = features.to(device)
        labels=labels.to(device)
        predictions= model(features)
        _,predicted=predictions.max(1,keepdim=True)
        print(predicted)
        correct+=torch.sum(predicted.view(-1,1)==labels.view(-1, 1)).item()
        
    accuracy = 100*correct/len(dataset.dataset)
    
        
    return accuracy


def train_step(model, model_0, mu:int, optimizer, train_data, loss_f):
    """Train `model` on one epoch of `train_data`"""
    
    total_loss=0
    
    for idx, (features,labels) in enumerate(train_data):
        
        optimizer.zero_grad()
        features = features.to(device)
        predictions= model(features)
        labels=labels.to(device)
        loss=loss_f(predictions,labels)
        loss+=mu/2*difference_models_norm_2(model,model_0)
        total_loss+=loss
        
        loss.backward()
    
        optimizer.step()
        
    return total_loss/(idx+1)



def local_learning(model, mu:float, optimizer, train_data, epochs:int, loss_f):
    
    model_0=deepcopy(model)
    
    for e in range(epochs):
        local_loss=train_step(model,model_0,mu,optimizer,train_data,loss_f)
        
    return float(local_loss.detach().cpu().numpy())


def difference_models_norm_2(model_1, model_2):
    """Return the norm 2 difference between the two model parameters
    """
    
    tensor_1=list(model_1.parameters())
    tensor_2=list(model_2.parameters())
    
    norm=sum([torch.sum((tensor_1[i]-tensor_2[i])**2) 
        for i in range(len(tensor_1))])
    
    return norm


def set_to_zero_model_weights(model):
    """Set all the parameters of a model to 0"""

    for layer_weigths in model.parameters():
        layer_weigths.data.sub_(layer_weigths.data)

In [141]:
def average_models(model, clients_models_hist:list , weights:list):

    """Creates the new model of a given iteration with the models of the other

    clients"""
    
    new_model=deepcopy(model)
    set_to_zero_model_weights(new_model)

    for k,client_hist in enumerate(clients_models_hist):
        
        for idx, layer_weights in enumerate(new_model.parameters()):

            contribution=client_hist[idx].data*weights[k]
            layer_weights.data.add_(contribution)
            
    return new_model
    

In [142]:
def FedProx(model, training_sets:list, n_iter:int, testing_sets:list, mu=0, 
    file_name="test", epochs=5, lr=10**-2):
    """ all the clients are considered in this implementation of FedProx
    Parameters:
        - `model`: common structure used by the clients and the server
        - `training_sets`: list of the training sets. At each index is the 
            training set of client "index"
        - `n_iter`: number of iterations the server will run
        - `testing_set`: list of the testing sets. If [], then the testing
            accuracy is not computed
        - `mu`: regularization term for FedProx. mu=0 for FedAvg
        - `epochs`: number of epochs each client is running
        - `lr`: learning rate of the optimizer
        - `decay`: to change the learning rate at each iteration
    
    returns :
        - `model`: the final global model 
    """
        
    loss_f=loss_classifier
    
    #Variables initialization
    K=len(training_sets) #number of clients
    n_samples=sum([len(db.dataset) for db in training_sets])
    weights=([len(db.dataset)/n_samples for db in training_sets])
    print("Clients' weights:",weights)
    
    
    loss_hist=[[float(loss_dataset(model, dl, loss_f).detach()) 
        for dl in training_sets]]
    acc_hist=[[accuracy_dataset(model, dl) for dl in testing_sets]]
    server_hist=[[tens_param.detach().cpu().numpy()
        for tens_param in list(model.parameters())]]
    models_hist = []
    
    
    server_loss=sum([weights[i]*loss_hist[-1][i] for i in range(len(weights))])
    server_acc=sum([weights[i]*acc_hist[-1][i] for i in range(len(weights))])
    print(f'====> i: 0 Loss: {server_loss} Server Test Accuracy: {server_acc}')
    server_loss_list=[]
    server_accuracy_list=[]  
    for i in range(n_iter):
        
        clients_params=[]
        clients_models=[]
        clients_losses=[]
        for k in range(K):
            local_model=deepcopy(model)
            local_optimizer=optim.SGD(local_model.parameters(),lr=0.001)
            
            local_loss=local_learning(local_model,mu,local_optimizer,
                training_sets[k],epochs,loss_f)
            
            clients_losses.append(local_loss)
                
            #GET THE PARAMETER TENSORS OF THE MODEL
            list_params=list(local_model.parameters())
            list_params=[tens_param.detach() for tens_param in list_params]
            clients_params.append(list_params)    
            clients_models.append(deepcopy(local_model))

            print(f"{k}---local_loss--- {local_loss}" )
        
        
        #CREATE THE NEW GLOBAL MODEL
        model = average_models(deepcopy(model), clients_params, 
            weights=weights)
        models_hist.append(clients_models)
        
        #COMPUTE THE LOSS/ACCURACY OF THE DIFFERENT CLIENTS WITH THE NEW MODEL
        loss_hist+=[[float(loss_dataset(model, dl, loss_f).detach()) 
            for dl in training_sets]]
        acc_hist+=[[accuracy_dataset(model, dl) for dl in testing_sets]]

        server_loss=sum([weights[i]*loss_hist[-1][i] for i in range(len(weights))])
        server_acc=sum([weights[i]*acc_hist[-1][i] for i in range(len(weights))])

        print(f'====> i: {i+1} Loss: {server_loss} Server Test Accuracy: {server_acc}')
        print(acc_hist)
        server_accuracy_list.append(server_acc)
        server_loss_list.append(server_loss)        

        server_hist.append([tens_param.detach().cpu().numpy() 
            for tens_param in list(model.parameters())])
        
        #DECREASING THE LEARNING RATE AT EACH SERVER ITERATION
            
    return model, loss_hist, acc_hist,server_accuracy_list,server_loss_list


In [143]:
n_iter=40

model_f, loss_hist_FA_iid, acc_hist_FA_iid,server_accuracy_list,server_loss_list = FedProx( model, 
    mnist_iid_train_dls, n_iter, mnist_iid_test_dls, epochs =3)

def plot_acc_loss(title:str, loss_hist:list, acc_hist:list):
    plt.figure()
    
    plt.suptitle(title)

    plt.subplot(1,2,1)
    lines=plt.plot(loss_hist)
    plt.title("Loss")
    plt.legend(lines,["C1", "C2", "C3","C4", "C5", "C6","C7", "C8", "C9","C10"])

    plt.subplot(1,2,2)
    lines=plt.plot(acc_hist )
    plt.title("Accuracy")
    plt.legend(lines,["C1", "C2", "C3","C4", "C5", "C6","C7", "C8", "C9","C10"])
    


import pickle


with open('acc-fedavo-lisa.pickle', 'wb') as handle:
    pickle.dump(acc_hist_FA_iid, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('loss-fedavo-lisa.pickle', 'wb') as handle:
    pickle.dump(loss_hist_FA_iid, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('server_acc_hist_fedavo-lisa.pickle', 'wb') as handle:
    pickle.dump(server_accuracy_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('server_loss_list_fedavo-lisa.pickle', 'wb') as handle:
    pickle.dump(server_loss_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
plot_acc_loss("FedAvg MNIST-AVOA-IID-Epoch-5-Client-10", loss_hist_FA_iid, acc_hist_FA_iid)


Clients' weights: [0.0154, 0.0224, 0.0186, 0.006, 0.033, 0.0136, 0.7686, 0.0534, 0.024, 0.045]
tensor([[11],
        [11],
        [11]], device='cuda:0')
tensor([[11],
        [11],
        [11],
        [11]], device='cuda:0')
tensor([[11],
        [11],
        [11],
        [11]], device='cuda:0')
tensor([[11],
        [11],
        [11],
        [11]], device='cuda:0')
tensor([[11],
        [11],
        [11],
        [11]], device='cuda:0')
tensor([[11],
        [11],
        [11],
        [11]], device='cuda:0')
tensor([[11],
        [11],
        [11],
        [11]], device='cuda:0')
tensor([[11],
        [11],
        [11],
        [11]], device='cuda:0')
tensor([[11],
        [11],
        [11],
        [11]], device='cuda:0')
tensor([[11],
        [11],
        [11],
        [11]], device='cuda:0')
tensor([[11],
        [11],
        [11],
        [11]], device='cuda:0')
tensor([[11],
        [11],
        [11],
        [11]], device='cuda:0')
tensor([[11],
        [11],
   

KeyboardInterrupt: 